In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
import csv
import pandas as pd

In [11]:
def crawler(symbol):
    driver = webdriver.Chrome("chromedriver") # change it to your own file road
    driver.get(url_form4)
    driver.find_element_by_id('cikform')
    driver.find_element_by_xpath("//input[@name='cik']").send_keys(symbol) 
    driver.find_element_by_xpath("//input[@value='Search!']").click()
    
    # click and jump to new windows，and remove handle
    winHandls = driver.window_handles
    driver.switch_to.window(winHandls[0])
    time.sleep(5)
    
    
    times = 0

    while True:
        try:
            # If there aren't period choice, no insider transaction. 
            select_option = driver.find_element_by_name("ChartPeriod")
        except NoSuchElementException as e:
            # End of searching for this company.
            print('{}: no insider transaction'.format(symbol))
            break
        
        # only need to change period to 15 years in the first page of a company
        if times == 0:
            times += 1
            all_options = select_option.find_elements_by_tag_name("option")

            # select time: 15 years
            for option in all_options:   
                if option.get_attribute("value") == '180':
                    option.click()
                    time.sleep(1)
            driver.find_element_by_xpath("//input[@value='Go!']").click()
            time.sleep(1)
        
        # if there aren't stock traded in this page, go to options
        try:
            table_tr_list_stock = driver.find_element(By.ID, 'filing_table').find_elements(By.TAG_NAME, "tr")

            # write detailed information in table
            for tr in table_tr_list_stock[1:]:    # go through every tr
                    
                # go through every td, write the row into a list
                table_td_list = tr.find_elements(By.TAG_NAME, "td")
                row_list = []
                for td in table_td_list:
                    row_list.append(td.text)    
                writer_stock.writerow(row_list)
        except NoSuchElementException as e:
            pass
        
        # if there aren't option traded in this page, go to next page
        try:
            table_tr_option = driver.find_element(By.ID, 'option_table').find_elements(By.TAG_NAME, "tr")
            
            # write detailed information in table
            for tr in table_tr_option[1:]:    
                
                # go through every td, write the row into a list
                table_td_list = tr.find_elements(By.TAG_NAME, "td")
                row_list = []
                for td in table_td_list:
                    row_list.append(td.text)
                writer_option.writerow(row_list)
        except NoSuchElementException as e:
            pass

        # make sure we reach the final page of a certain company
        try:    
            driver.execute_script("return arguments[0].scrollIntoView(true);", WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.XPATH, "//a[contains(text(),'Next')]"))))
            driver.find_element_by_xpath("//a[contains(text(),'Next')]").click()

        except (TimeoutException, WebDriverException) as e:
            break
    driver.close()
    
        

In [13]:
url_form4 = 'https://www.secform4.com/'
outfile1 = open('sec_form4_stock.csv','w',newline = '')
outfile2 = open('sec_form4_option.csv','w',newline = '')

# write in stock table
title_stock = ['Transaction Date','Reported DateTime','Company','Symbol','Insider Relationship','Shares Traded','Average Price','Total Amount','Shares Owned','Filing']
title_option = ['Transaction Date','Reported DateTime','Exercisable Expiration','Company','Symbol','Insider Relationship','Shares Traded','Conversion Price','Shares Owned','Filing']
writer_stock = csv.writer(outfile1)
writer_option = csv.writer(outfile2)

# Insert a header
writer_stock.writerow(title_stock)
writer_option.writerow(title_option)
sp500 = pd.read_csv('s&p500.CSV')
for i in sp500['CIK']:
    crawler(i)
    
outfile1.close()
outfile2.close()  

1132979: no insider transaction
